In [0]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType




customer data


In [0]:
from pyspark.sql.types import StructType, StructField, StringType
customer_schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("customer_unique_id", StringType(), True),
    StructField("customer_zip_code_prefix", StringType(), True),
    StructField("customer_city", StringType(), True),
    StructField("customer_state", StringType(), True)
])

df_customers = spark.read.schema(customer_schema).table("workspace.ecommerce.olist_customers_dataset")
display(df_customers.head(5))
# display(df_customers.describe())
is_null_customer = {col_name: df_customers.filter(df_customers[col_name].isNull()).count() for col_name in df_customers.columns}
percentage_of_null_customer = {col_name: (df_customers.filter(df_customers[col_name].isNull()).count() / df_customers.count()) * 100 for col_name in df_customers.columns}

df_customers.printSchema()

display({col_name: f"{percentage:.2f} %" for col_name, percentage in percentage_of_null_customer.items()})
print("\n")
display(is_null_customer)


customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: long (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



{'customer_id': '0.00 %',
 'customer_unique_id': '0.00 %',
 'customer_zip_code_prefix': '0.00 %',
 'customer_city': '0.00 %',
 'customer_state': '0.00 %'}

{'customer_id': 0,
 'customer_unique_id': 0,
 'customer_zip_code_prefix': 0,
 'customer_city': 0,
 'customer_state': 0}

order data

In [0]:

order_schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("order_status",StringType(),True),
    StructField("order_purchase_timestamp", StringType(), True),
    StructField("order_approved_at", StringType(), True),
    StructField("order_delivered_carrier_date", StringType(), True),
    StructField("order_delivered_customer_date", StringType(), True),
    StructField("order_estimated_delivery_date", StringType(), True)
])

df_orders = spark.read.schema(order_schema).table("workspace.ecommerce.olist_orders_dataset")
df_orders.printSchema()

display(df_orders.head(5))
display(df_orders.describe())
is_null_order = {col_name: df_orders.filter(df_orders[col_name].isNull()).count() for col_name in df_orders.columns}
percentage_of_null_order = {col_name: (df_orders.filter(df_orders[col_name].isNull()).count() / df_orders.count()) * 100 for col_name in df_orders.columns}

display({col_name: f"{percentage:.2f} %" for col_name, percentage in percentage_of_null_order.items()})
print("\n")
display(is_null_order)

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02T10:56:33.000Z,2017-10-02T11:07:15.000Z,2017-10-04T19:55:00.000Z,2017-10-10T21:25:13.000Z,2017-10-18T00:00:00.000Z
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24T20:41:37.000Z,2018-07-26T03:24:27.000Z,2018-07-26T14:31:00.000Z,2018-08-07T15:27:45.000Z,2018-08-13T00:00:00.000Z
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08T08:38:49.000Z,2018-08-08T08:55:23.000Z,2018-08-08T13:50:00.000Z,2018-08-17T18:06:29.000Z,2018-09-04T00:00:00.000Z
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18T19:28:06.000Z,2017-11-18T19:45:59.000Z,2017-11-22T13:39:59.000Z,2017-12-02T00:28:42.000Z,2017-12-15T00:00:00.000Z
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13T21:18:39.000Z,2018-02-13T22:20:29.000Z,2018-02-14T19:46:34.000Z,2018-02-16T18:17:02.000Z,2018-02-26T00:00:00.000Z


summary,order_id,customer_id,order_status
count,99441,99441,99441
mean,null,null,null
stddev,null,null,null
min,00010242fe8c5a6d1ba2dd792cb16214,00012a2ce6f8dcda20d059ce98491703,approved
max,fffe41c64501cc87c801fd61db3f6244,ffffe8b65bbe3087b653a978c870db99,unavailable


{'order_id': '0.00 %',
 'customer_id': '0.00 %',
 'order_status': '0.00 %',
 'order_purchase_timestamp': '0.00 %',
 'order_approved_at': '0.16 %',
 'order_delivered_carrier_date': '1.79 %',
 'order_delivered_customer_date': '2.98 %',
 'order_estimated_delivery_date': '0.00 %'}

{'order_id': 0,
 'customer_id': 0,
 'order_status': 0,
 'order_purchase_timestamp': 0,
 'order_approved_at': 160,
 'order_delivered_carrier_date': 1783,
 'order_delivered_customer_date': 2965,
 'order_estimated_delivery_date': 0}

order items

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

order_items_schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("order_item_id", StringType(), True),
    StructField("product_id", StringType(), True),
    StructField("seller_id", StringType(), True),
    StructField("shipping_limit_date", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("freight_value", DoubleType(), True)
])

df_order_items = spark.read.schema(order_items_schema).table("workspace.ecommerce.olist_order_items_dataset")
df_order_items.printSchema()
display(df_order_items.head(5))
is_null_order_items = {col_name: df_order_items.filter(df_order_items[col_name].isNull()).count() for col_name in df_order_items.columns}
percentage_of_null_order_items = {col_name: (df_order_items.filter(df_order_items[col_name].isNull()).count() / df_order_items.count()) * 100 for col_name in df_order_items.columns}

display({col_name: f"{percentage:.2f} %" for col_name, percentage in percentage_of_null_order_items.items()})
print("\n")
display(is_null_order_items)
display(df_order_items.describe())

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: long (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)



order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19T09:45:35.000Z,58.9,13.29
00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03T11:05:13.000Z,239.9,19.93
000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18T14:48:30.000Z,199.0,17.87
00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15T10:10:18.000Z,12.99,12.79
00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13T13:57:51.000Z,199.9,18.14


{'order_id': '0.00 %',
 'order_item_id': '0.00 %',
 'product_id': '0.00 %',
 'seller_id': '0.00 %',
 'shipping_limit_date': '0.00 %',
 'price': '0.00 %',
 'freight_value': '0.00 %'}

{'order_id': 0,
 'order_item_id': 0,
 'product_id': 0,
 'seller_id': 0,
 'shipping_limit_date': 0,
 'price': 0,
 'freight_value': 0}

summary,order_id,order_item_id,product_id,seller_id,price,freight_value
count,112650,112650,112650,112650,112650,112650
mean,null,1.1978339991122948,null,null,120.65373901477311,19.99031992898562
stddev,null,0.7051240313951734,null,null,183.63392805025924,15.806405412297073
min,00010242fe8c5a6d1ba2dd792cb16214,1,00066f42aeeb9f3007548bb9d3f33c38,0015a82c2db000af6aaaf3ae2ecb0532,0.85,0.0
max,fffe41c64501cc87c801fd61db3f6244,21,fffe9eeff12fcbd74a2f2b007dde0c58,ffff564a4f9085cd26170f4732393726,6735.0,409.68


order payments

In [0]:

from pyspark.sql.types import StructType, StructField, StringType, DoubleType,IntegerType

order_payments_schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("payment_sequential", StringType(), True),
    StructField("payment_type", StringType(), True),
    StructField("payment_installments", IntegerType(), True),
    StructField("payment_value", DoubleType(), True)
])

df_order_payments = spark.read.schema(order_payments_schema).table("workspace.ecommerce.olist_order_payments_dataset")
df_order_payments.printSchema()

display(df_order_payments.head(5))
is_null_order_payments = {col_name: df_order_payments.filter(df_order_payments[col_name].isNull()).count() for col_name in df_order_payments.columns}
percentage_of_null_order_payments = {col_name: (df_order_payments.filter(df_order_payments[col_name].isNull()).count() / df_order_payments.count() * 100) for col_name in df_order_payments.columns}
display(is_null_order_payments)
print("\n")
display({col_name:f"{percentage:.2f} %" for col_name, percentage in percentage_of_null_order_payments.items()})
display(df_order_payments.describe())


root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: long (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: long (nullable = true)
 |-- payment_value: double (nullable = true)



order_id,payment_sequential,payment_type,payment_installments,payment_value
b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


{'order_id': 0,
 'payment_sequential': 0,
 'payment_type': 0,
 'payment_installments': 0,
 'payment_value': 0}

{'order_id': '0.00 %',
 'payment_sequential': '0.00 %',
 'payment_type': '0.00 %',
 'payment_installments': '0.00 %',
 'payment_value': '0.00 %'}

summary,order_id,payment_sequential,payment_type,payment_installments,payment_value
count,103886,103886,103886,103886,103886
mean,null,1.0926785129853878,null,2.853348863176944,154.10038041698365
stddev,null,0.7065837791949956,null,2.6870506738564788,217.4940638647242
min,00010242fe8c5a6d1ba2dd792cb16214,1,boleto,0,0.0
max,fffe41c64501cc87c801fd61db3f6244,29,voucher,24,13664.08


order reviews


In [0]:

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

order_reviews_schema = StructType([
    StructField("review_id", StringType(), True),
    StructField("order_id", StringType(), True),
    StructField("review_score", IntegerType(), True),
    StructField("review_comment_title", StringType(), True),
    StructField("review_comment_message", StringType(), True),
    StructField("review_creation_date", StringType(), True),
    StructField("review_answer_timestamp", StringType(), True)
])

df_order_reviews = spark.read.schema(order_reviews_schema).table("workspace.ecommerce.olist_order_reviews_dataset")
df_order_reviews.printSchema()

display(df_order_reviews.limit(5))
is_null_order_reviews = {col_name: df_order_reviews.filter(df_order_reviews[col_name].isNull()).count() for col_name in df_order_reviews.columns}
percentage_of_null_order_reviews = {col_name: (df_order_reviews.filter(df_order_reviews[col_name].isNull()).count() / df_order_reviews.count() * 100) for col_name in df_order_reviews.columns}
display(is_null_order_reviews)
print("\n")
display({col_name:f"{percentage:.2f} %" for col_name, percentage in percentage_of_null_order_reviews.items()})
display(df_order_reviews.describe())

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: long (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: timestamp (nullable = true)
 |-- review_answer_timestamp: timestamp (nullable = true)



review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,null,null,2018-01-18T00:00:00.000Z,2018-01-18T21:46:59.000Z
80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,null,null,2018-03-10T00:00:00.000Z,2018-03-11T03:05:13.000Z
228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,null,null,2018-02-17T00:00:00.000Z,2018-02-18T14:36:24.000Z
e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,null,Recebi bem antes do prazo estipulado.,2017-04-21T00:00:00.000Z,2017-04-21T22:02:06.000Z
f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,null,Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,2018-03-01T00:00:00.000Z,2018-03-02T10:26:53.000Z


{'review_id': 1,
 'order_id': 2236,
 'review_score': 4937,
 'review_comment_title': 92157,
 'review_comment_message': 63079,
 'review_creation_date': 8767,
 'review_answer_timestamp': 8785}

{'review_id': '0.00 %',
 'order_id': '2.15 %',
 'review_score': '4.74 %',
 'review_comment_title': '88.47 %',
 'review_comment_message': '60.56 %',
 'review_creation_date': '8.42 %',
 'review_answer_timestamp': '8.43 %'}

summary,review_id,order_id,review_score,review_comment_title,review_comment_message
count,104161,101926,99225,12005,41083
mean,4.5,0.0,4.086379440665155,3.165434995880696E10,8.172413793103448
stddev,0.7071067811865476,0.0,1.3476347819432772,5.625434750908947E11,3.1175650470615848
min,"""",,0,,
max,"🤙🏼👏🏼👏🏼""",visando sempre o melhor para os clientes!,5,🔟,😡😡😡😡😡👎👎👎👎👎


products

In [0]:

from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

products_schema = StructType([
    StructField("product_id", StringType(), True),
    StructField("product_category_name", StringType(), True),
    StructField("product_name_lenght", IntegerType(), True),
    StructField("product_description_lenght", IntegerType(), True),
    StructField("product_photos_qty", IntegerType(), True),
    StructField("product_weight_g", DoubleType(), True),
    StructField("product_length_cm", DoubleType(), True),
    StructField("product_height_cm", DoubleType(), True),
    StructField("product_width_cm", DoubleType(), True)
])

df_products = spark.read.schema(products_schema).table("workspace.ecommerce.olist_products_dataset")
df_products.printSchema()
display(df_products.limit(5))
is_null_products = {col_name: df_products.filter(df_products[col_name].isNull()).count() for col_name in df_products.columns}
percentage_of_null_products = {col_name: (df_products.filter(df_products[col_name].isNull()).count() / df_products.count() * 100) for col_name in df_products.columns}
display(is_null_products)
print("\n")
display({col_name:f"{percentage:.2f} %" for col_name, percentage in percentage_of_null_products.items()})
display(df_products.describe())

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: long (nullable = true)
 |-- product_description_lenght: long (nullable = true)
 |-- product_photos_qty: long (nullable = true)
 |-- product_weight_g: long (nullable = true)
 |-- product_length_cm: long (nullable = true)
 |-- product_height_cm: long (nullable = true)
 |-- product_width_cm: long (nullable = true)



product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40,287,1,225,16,10,14
3aa071139cb16b67ca9e5dea641aaa2f,artes,44,276,1,1000,30,18,20
96bd76ec8810374ed1b65e291975717f,esporte_lazer,46,250,1,154,18,9,15
cef67bcfe19066a932b7673e239eb23d,bebes,27,261,1,371,26,4,26
9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37,402,4,625,20,17,13


{'product_id': 0,
 'product_category_name': 610,
 'product_name_lenght': 610,
 'product_description_lenght': 610,
 'product_photos_qty': 610,
 'product_weight_g': 2,
 'product_length_cm': 2,
 'product_height_cm': 2,
 'product_width_cm': 2}

{'product_id': '0.00 %',
 'product_category_name': '1.85 %',
 'product_name_lenght': '1.85 %',
 'product_description_lenght': '1.85 %',
 'product_photos_qty': '1.85 %',
 'product_weight_g': '0.01 %',
 'product_length_cm': '0.01 %',
 'product_height_cm': '0.01 %',
 'product_width_cm': '0.01 %'}

summary,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32951,32341,32341,32341,32341,32949,32949,32949,32949
mean,null,null,48.47694876472589,771.4952846232337,2.1889861166939797,2276.4724877841513,30.81507784758263,16.937661234028347,23.196728277034204
stddev,null,null,10.245740725237221,635.1152246349536,1.7367656379315475,4282.038730976971,16.91445805406594,13.637554061749485,12.079047453227828
min,00066f42aeeb9f3007548bb9d3f33c38,agro_industria_e_comercio,5,4,1,0,7,2,6
max,fffe9eeff12fcbd74a2f2b007dde0c58,utilidades_domesticas,76,3992,20,40425,105,105,118


sellers


In [0]:

from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

sellers_schema = StructType([
    StructField("seller_id", StringType(), True),
    StructField("seller_zip_code_prefix", StringType(), True),
    StructField("seller_city", StringType(), True),
    StructField("seller_state", StringType(), True)
])

df_sellers = spark.read.schema(sellers_schema).table("workspace.ecommerce.olist_sellers_dataset")
df_sellers.printSchema()

display(df_sellers.limit(5))
is_null_sellers = {col_name: df_sellers.filter(df_sellers[col_name].isNull()).count() for col_name in df_sellers.columns}
percentage_of_null_sellers = {col_name: round((count/df_sellers.count())*100, 2) for col_name, count in is_null_sellers.items()}
display(is_null_sellers)
print("\n")
display({col_name:f"{percentage:.2f} %" for col_name, percentage in percentage_of_null_sellers.items()})
display(df_sellers.describe())


root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: long (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)



seller_id,seller_zip_code_prefix,seller_city,seller_state
3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


{'seller_id': 0,
 'seller_zip_code_prefix': 0,
 'seller_city': 0,
 'seller_state': 0}

{'seller_id': '0.00 %',
 'seller_zip_code_prefix': '0.00 %',
 'seller_city': '0.00 %',
 'seller_state': '0.00 %'}

summary,seller_id,seller_zip_code_prefix,seller_city,seller_state
count,3095,3095,3095,3095
mean,null,32291.059450726978,4482255.0,null
stddev,null,32713.45382950907,null,null
min,0015a82c2db000af6aaaf3ae2ecb0532,1001,04482255,AC
max,ffff564a4f9085cd26170f4732393726,99730,xaxim,SP


geo location


In [0]:

from pyspark.sql.types import StructType, StructField, StringType, DoubleType

geolocation_schema = StructType([
    StructField("geolocation_zip_code_prefix", StringType(), True),
    StructField("geolocation_lat", DoubleType(), True),
    StructField("geolocation_lng", DoubleType(), True),
    StructField("geolocation_city", StringType(), True),
    StructField("geolocation_state", StringType(), True)
])

df_geolocation = spark.read.schema(geolocation_schema).table("workspace.ecommerce.olist_geolocation_dataset")
df_geolocation.printSchema()
display(df_geolocation.limit(5))
is_null_geolocation = {col_name: df_geolocation.filter(df_geolocation[col_name].isNull()).count() for col_name in df_geolocation.columns}
percentage_of_null_geolocation = {col_name: (df_geolocation.filter(df_geolocation[col_name].isNull()).count() / df_geolocation.count() * 100) for col_name in df_geolocation.columns}
display(is_null_geolocation)
print("\n")
display({col_name:f"{percentage:.2f} %" for col_name, percentage in percentage_of_null_geolocation.items()})
display(df_geolocation.describe())


root
 |-- geolocation_zip_code_prefix: long (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)



geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
1037,-23.54562128115268,-46.63929204800168,sao paulo,SP
1046,-23.546081127035535,-46.64482029837157,sao paulo,SP
1046,-23.54612896641469,-46.64295148361138,sao paulo,SP
1041,-23.5443921648681,-46.63949930627844,sao paulo,SP
1035,-23.541577961711493,-46.64160722329613,sao paulo,SP


{'geolocation_zip_code_prefix': 0,
 'geolocation_lat': 0,
 'geolocation_lng': 0,
 'geolocation_city': 0,
 'geolocation_state': 0}

{'geolocation_zip_code_prefix': '0.00 %',
 'geolocation_lat': '0.00 %',
 'geolocation_lng': '0.00 %',
 'geolocation_city': '0.00 %',
 'geolocation_state': '0.00 %'}

summary,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
count,1000163,1000163,1000163,1000163,1000163
mean,36574.16646586607,-21.176152910383102,-46.390541320935995,null,null
stddev,30549.335710319865,5.715866308822856,4.2697483066196,null,null
min,1001,-36.6053744107061,-101.46676644931476,* cidade,AC
max,99990,45.06593318269697,121.10539381057764,óleo,TO


products category translation

In [0]:
from pyspark.sql.types import StructType, StructField, StringType

products_category_schema = StructType([
    StructField("product_category_name", StringType(), True),
    StructField("product_category_name_english", StringType(), True)
])

df_products_category = spark.read.schema(products_category_schema).table("workspace.ecommerce.product_category_name_translation")
df_products_category.printSchema()
display(df_products_category.limit(5))
is_null_products_category = {col_name: df_products_category.filter(df_products_category[col_name].isNull()).count() for col_name in df_products_category.columns}
percentage_of_null_products_category = {col_name: (df_products_category.filter(df_products_category[col_name].isNull()).count() / df_products_category.count() * 100) for col_name in df_products_category.columns}
display(is_null_products_category)
print("\n")
display({col_name:f"{percentage:.2f} %" for col_name, percentage in percentage_of_null_products_category.items()})
display(df_products_category.describe())

root
 |-- product_category_name: string (nullable = true)
 |-- product_category_name_english: string (nullable = true)



product_category_name,product_category_name_english
beleza_saude,health_beauty
informatica_acessorios,computers_accessories
automotivo,auto
cama_mesa_banho,bed_bath_table
moveis_decoracao,furniture_decor


{'product_category_name': 0, 'product_category_name_english': 0}

{'product_category_name': '0.00 %', 'product_category_name_english': '0.00 %'}

summary,product_category_name,product_category_name_english
count,71,71
mean,null,null
stddev,null,null
min,agro_industria_e_comercio,agro_industry_and_commerce
max,utilidades_domesticas,watches_gifts


In [0]:

def indian_number_format(num):
    try:
        num = float(num)
        if num < 1000:
            return f"{num:.2f}"
        elif num < 100000:
            return f"{num/1000:.2f} Thousand"
        elif num < 10000000:
            return f"{num/100000:.2f} Lakh"
        else:
            return f"{num/10000000:.2f} Crore"
    except:
        return None

indian_format_udf = udf(indian_number_format, StringType())


product_category_name_english,total_weight,total_weight_formatted
furniture_decor,7992965,79.93 Lakh
bed_bath_table,7440451,74.40 Lakh
housewares,7053553,70.54 Lakh
sports_leisure,5804731,58.05 Lakh
auto,5043836,50.44 Lakh
office_furniture,3936928,39.37 Lakh
health_beauty,3506636,35.07 Lakh
baby,3355475,33.55 Lakh
toys,2637662,26.38 Lakh
garden_tools,2337144,23.37 Lakh


**Handling Nulls in Orders (df_orders)**

The Problem: You have nulls in order_delivered_customer_date (2.98%) and order_approved_at (0.16%).

The Logic: If a delivery date is missing, the order might still be in progress or canceled.

The Fix:

For Delivered analysis: Drop rows where the delivery date is null.

For Funnel analysis: Keep them but flag them as "Undelivered".

In [0]:
from pyspark.sql.functions import col, when, datediff, round,count

# 1. Create a "Clean" orders dataframe for delivery analysis
# We remove orders that were never delivered/canceled to calculate accurate delivery times
df_orders_clean = df_orders.dropna(subset=["order_delivered_customer_date"])

# 2. Fill nulls in 'order_approved_at' with the purchase timestamp
# (Assumption: Instant approval if missing)
df_orders_filled = df_orders.withColumn(
    "order_approved_at",
    when(col("order_approved_at").isNull(), col("order_purchase_timestamp"))
    .otherwise(col("order_approved_at"))
)

**Handling Nulls in Reviews (df_order_reviews)**

The Problem: review_comment_title is 88% empty and review_comment_message is 60% empty.

The Logic: This is normal. People rate stars (score) but don't always write text.

The Fix: Replace nulls with "No Comment" so your text analysis code doesn't crash.

In [0]:
df_reviews_clean = df_order_reviews.fillna({
    "review_comment_title": "No Title",
    "review_comment_message": "No Message"
})

**Handling Product Categories (df_products)**

The Problem: product_category_name has 1.85% nulls.

The Fix: Fill with "unknown". Also, join with df_products_category to get the English names, as the original names are likely in Portuguese.

In [0]:
# 1. Fill unknown categories
df_products_clean = df_products.fillna({"product_category_name": "unknown"})

# 2. Join to get English names (Crucial for presentation)
df_products_enriched = df_products_clean.join(
    df_products_category, 
    "product_category_name", 
    "left"
).select(
    df_products_clean["*"],
    # If English name is missing (for 'unknown' category), use 'Unknown'
    when(col("product_category_name_english").isNull(), "Unknown")
    .otherwise(col("product_category_name_english")).alias("category_name")
)

**Analysis A: Delivery Performance (Logistics)**

# Business Question: "How long does it really take to deliver our products? Are we late?"

In [0]:

from pyspark.sql.functions import col, when, datediff, round,count,avg
# Calculate "Actual Delivery Days" vs "Estimated Delivery Days"
delivery_performance = df_orders_clean.withColumn(
    "actual_days", 
    datediff(col("order_delivered_customer_date"), col("order_purchase_timestamp"))
).withColumn(
    "estimated_days", 
    datediff(col("order_estimated_delivery_date"), col("order_purchase_timestamp"))
).withColumn(
    "is_late", 
    when(col("actual_days") > col("estimated_days"), 1).otherwise(0)
)

# Aggregated Metrics
delivery_metrics = delivery_performance.agg(
    round(avg("actual_days"), 2).alias("avg_delivery_days"),
    round(avg("estimated_days"), 2).alias("avg_estimated_days"),
    sum("is_late").alias("total_late_orders"),
    count("*").alias("total_orders")
).withColumn(
    "late_percentage", 
    round((col("total_late_orders") / col("total_orders")) * 100, 2)
)

display(delivery_metrics)

avg_delivery_days,avg_estimated_days,total_late_orders,total_orders,late_percentage
12.5,24.37,6535,96476,6.77


**Top Performing Product Categories (Sales)**

# Business Question: "Which categories generate the most revenue?"

Requires Joining: Order Items -> Products

In [0]:
# Join items with products to get category names
revenue_by_category = df_order_items.join(df_products_enriched, "product_id") \
    .groupBy("category_name") \
    .agg(
        round(sum("price"), 2).alias("total_revenue"),
        count("order_id").alias("total_orders")
    ) \
    .withColumn("total_revenue_formatted", indian_format_udf(col("total_revenue"))) \
    .orderBy(col("total_revenue").desc())

display(revenue_by_category)

category_name,total_revenue,total_orders,total_revenue_formatted
health_beauty,1258681.34,9670,12.59 Lakh
watches_gifts,1205005.68,5991,12.05 Lakh
bed_bath_table,1036988.68,11115,10.37 Lakh
sports_leisure,988048.97,8641,9.88 Lakh
computers_accessories,911954.32,7827,9.12 Lakh
furniture_decor,729762.49,8334,7.30 Lakh
cool_stuff,635290.85,3796,6.35 Lakh
housewares,632248.66,6964,6.32 Lakh
auto,592720.11,4235,5.93 Lakh
garden_tools,485256.46,4347,4.85 Lakh


**Customer Value (RFM Proxy)**

# Business Question: "Who are our big spenders? (States)"

Requires Joining: Orders -> Order Items -> Customers

In [0]:
# 1. Get total spend per order
order_spend = df_order_items.groupBy("order_id").agg(sum("price").alias("order_total"))

# 2. Join with Customers to find location
state_revenue = df_orders.join(order_spend, "order_id") \
    .join(df_customers, "customer_id") \
    .groupBy("customer_state") \
    .agg(
        round(sum("order_total"), 2).alias("total_revenue"),
        count("order_id").alias("order_count")
    ) \
    .withColumn("total_revenue_formatted", indian_format_udf(col("total_revenue")))\
    .orderBy(col("total_revenue").desc())

display(state_revenue)

customer_state,total_revenue,order_count,total_revenue_formatted
SP,5202955.05,41375,52.03 Lakh
RJ,1824092.67,12762,18.24 Lakh
MG,1585308.03,11544,15.85 Lakh
RS,750304.02,5432,7.50 Lakh
PR,683083.76,4998,6.83 Lakh
SC,520553.34,3612,5.21 Lakh
BA,511349.99,3358,5.11 Lakh
DF,302603.94,2125,3.03 Lakh
GO,294591.95,2007,2.95 Lakh
ES,275037.31,2025,2.75 Lakh


**Map the Sellers vs. Customers.(Geospatial Analysis)**

# Business Question: "Are sellers located near our customers, or are we shipping across the country?"

Requires Joining: df_geolocation -> Customers

In [0]:
# Clean Geolocation: Take the average Lat/Lng for each Zip Code to avoid duplicates
df_geo_clean = df_geolocation.groupBy("geolocation_zip_code_prefix") \
    .agg(
        avg("geolocation_lat").alias("lat"),
        avg("geolocation_lng").alias("lng")
    )

# Now you can join this 'df_geo_clean' with 'df_customers' (on zip) 
df_customers_geo = df_customers.join(
    df_geo_clean,
    df_customers.customer_zip_code_prefix == df_geo_clean.geolocation_zip_code_prefix,
    "left"
)

# and 'df_sellers' (on zip) to visualize locations.
df_sellers_geo = df_sellers.join(
    df_geo_clean,
    df_sellers.seller_zip_code_prefix == df_geo_clean.geolocation_zip_code_prefix,
    "left"
)

display(df_customers_geo.limit(5))
display(df_sellers_geo.limit(5))

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,lat,lng
06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,14409,-20.498488755380297,-47.39692948590099
18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,9790,-23.72799221530055,-46.542847789990844
4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,1151,-23.53164158468372,-46.65628875324909
b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,8775,-23.49970230179501,-46.185232958890026
4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,13056,-22.975100456164604,-47.142925230302346


seller_id,seller_zip_code_prefix,seller_city,seller_state,geolocation_zip_code_prefix,lat,lng
3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,13023,-22.893848032534073,-47.06133702244192
d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP,13844,-22.383436514042817,-46.94792654261969
ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ,20031,-22.90957243765554,-43.177703112986926
c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP,4195,-23.65724198521616,-46.612830839593734
51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP,12914,-22.964802783097724,-46.534419315285234


**RFM Analysis (Customer Segmentation)**

Goal: Segment customers into "Champions," "At Risk," and "Hibernating" based on their buying behavior.

Recency: How many days since their last purchase?

Frequency: How many total orders have they made?

Monetary: How much have they spent in total?

Requires joining df_orders, df_order_items, and df_customers.

In [0]:
from pyspark.sql.functions import max, countDistinct, sum, datediff, current_date, col, lit

# 1. Prepare the Base Table
# We need Customer Unique ID, Last Purchase Date, Total Orders, Total Spend
rfm_table = df_orders.join(df_order_items, "order_id") \
    .join(df_customers, "customer_id") \
    .groupBy("customer_unique_id") \
    .agg(
        max("order_purchase_timestamp").alias("last_purchase_date"),
        countDistinct("order_id").alias("frequency"),
        sum("price").alias("monetary")
    ) \
    .withColumn("recency", datediff(current_date(), col("last_purchase_date")))

# 2. Score them (Simple Quantile-based scoring)
# Create simple segments (Logic: High Recency is BAD, High Freq/Monetary is GOOD)
rfm_scored = rfm_table.withColumn(
    "segment", 
    when((col("recency") < 90) & (col("frequency") >= 2), "Champion")
    .when((col("recency") < 90) & (col("frequency") == 1), "New Customer")
    .when((col("recency") >= 90) & (col("monetary") > 500), "Can't Lose")
    .otherwise("Hibernating")
)

display(rfm_scored.groupBy("segment").count())

segment,count
Can't Lose,3753
Hibernating,91667


**Cohort Analysis (Retention)**

Goal: See if customers acquired in January stick around longer than those acquired in February. 

Metric: Retention Rate (Month 0 to Month 12).

Requires--> df_orders and df_customers.

In [0]:
from pyspark.sql.functions import date_format, min, month, year

# 1. Get the "Cohort Month" (First Purchase Month) for each customer
customer_cohorts = df_orders.join(df_customers, "customer_id") \
    .groupBy("customer_unique_id") \
    .agg(min("order_purchase_timestamp").alias("first_purchase_date")) \
    .withColumn("cohort_month", date_format(col("first_purchase_date"), "yyyy-MM"))

# 2. Join back to get ALL activities
cohort_analysis = df_orders.join(df_customers, "customer_id") \
    .join(customer_cohorts, "customer_unique_id") \
    .withColumn("activity_month", date_format(col("order_purchase_timestamp"), "yyyy-MM")) \
    .groupBy("cohort_month", "activity_month") \
    .agg(countDistinct("customer_unique_id").alias("active_users")) \
    .orderBy("cohort_month", "activity_month")

# This result is best viewed in a Pivot Table in Excel/Pandas
display(cohort_analysis)

cohort_month,activity_month,active_users
2016-09,2016-09,4
2016-10,2016-10,321
2016-10,2017-04,1
2016-10,2017-07,1
2016-10,2017-09,1
2016-10,2017-11,1
2016-10,2018-01,1
2016-10,2018-03,1
2016-10,2018-05,2
2016-10,2018-06,2
